In [29]:
def init_seq(rows, cols):

    M = []

    for r in range(0, rows):
        arr = []
        for c in range(0, cols):
            arr.append(float(r * cols + c))
        M.append(arr)

    return M

In [30]:
r = 4
c = 4

init_seq(r, c)

[[0.0, 1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0, 7.0],
 [8.0, 9.0, 10.0, 11.0],
 [12.0, 13.0, 14.0, 15.0]]